In [ ]:
import numpy as np
import networkx as nx
from typing import Any, Dict, Hashable, List, Tuple, Callable
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Union, List, Tuple, Dict, Any, Hashable, Callable
import numpy as np
import networkx as nx
from node2vec import Node2Vec
from community import community_louvain  # python-louvain package
from collections import defaultdict
from itertools import combinations
import matplotlib.pyplot as plt
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

def l2_distance(vec1, vec2):
    """
    Computes the L2 norm (Euclidean distance) between two 128-dimensional vectors.

    Parameters:
        vec1 (np.ndarray): First vector, shape (128,)
        vec2 (np.ndarray): Second vector, shape (128,)

    Returns:
        float: L2 distance between vec1 and vec2
    """
    return np.linalg.norm(vec1 - vec2)


def a_star_search(graph: nx.Graph, source: Any, destination: Any, heuristic: Callable[[Any, Any], float] = l2_distance):
    """
    Runs the A* algorithm on a NetworkX graph with a 2-arg heuristic.

    Parameters:
    - graph: A NetworkX graph.
    - source: The starting node.
    - destination: The goal node.
    - heuristic: A function h(n, goal) that estimates the cost from node n to the goal.

    Returns:
    - path: List of nodes representing the shortest path.
    - cost: Total cost of the path.
    """
    def heuristic_wrapper(n1, n2):
        # Use destination from outer scope, ignore n2 parameter
        return heuristic(n1, destination)

    try:
        path = nx.astar_path(graph, source, destination, heuristic=heuristic_wrapper)
        cost = nx.path_weight(graph, path, weight='weight')
        return path, cost
    except nx.NetworkXNoPath:
        return None, float('inf')


class EmbeddedCommunity:
    def __init__(self, graph: nx.Graph, bounds):
        """
        Initializes the EmbeddedGraph with a NetworkX graph.

        Args:
            graph (nx.Graph): Input graph to embed and decompose into communities.
            bounds: Boundary nodes for this community
        """
        self.graph = graph.copy()  # Create a copy to avoid modifying the original
        self.community_bounds = bounds
        self.node_embeddings = {}

    def _enumerate_paths(self,source):
      distance_to_bounds = []
      for b in self.community_bounds:
        distance_to_bounds.append(l2_distance(self.node_embeddings[source],self.node_embeddings[b]))

      return distance_to_bounds



    def embed(self) -> Dict[Hashable, np.ndarray]:
        """
        Runs node2vec embedding on the graph and creates an embedding for each node.

        Returns:
            Dict[Hashable, np.ndarray]: Mapping from node ID to embedding vector.
        """
        embedded_nodes = defaultdict(int)
        embedding_dim = 128  # Default embedding dimension

        # Extract the subgraph for this community
        subgraph = self.graph

        # Skip communities that are too small for meaningful embedding
        if len(subgraph.nodes) < 2:
            # For isolated nodes, create a random embedding
            for node in subgraph.nodes():
                embedding = np.random.normal(0, 0.1, embedding_dim)
                embedded_nodes[node] = embedding
            return embedded_nodes

        try:
            # Configure node2vec
            node2vec = Node2Vec(
                subgraph,
                dimensions=embedding_dim,
                walk_length=30,
                num_walks=200,
                workers=4,
                p=1,  # Return parameter
                q=1,  # In-out parameter
            )

            # Train the model
            model = node2vec.fit(window=10, min_count=1)

            # Extract embeddings for each node
            for node in subgraph.nodes():
                try:
                    # Convert node to string for node2vec lookup
                    node_str = str(node)
                    embedding = np.array(model.wv[node_str])
                    embedded_nodes[node] = embedding
                except KeyError:
                    # Handle nodes that might not have been embedded properly
                    embedding = np.random.normal(0, 0.1, embedding_dim)
                    embedded_nodes[node] = embedding
        except Exception as e:
            # Fallback: Create random embeddings if node2vec fails
            for node in subgraph.nodes():
                embedding = np.random.normal(0, 0.1, embedding_dim)
                embedded_nodes[node] = embedding

        self.node_embeddings = embedded_nodes
        return embedded_nodes

    def get_top_k_boundary_nodes(self, k=5) -> List[Hashable]:
        """
        Selects top-k boundary nodes using degree, PageRank, and L2 to mean embedding.

        Returns:
            List[Hashable]: Top-k nodes.
        """
        candidates = set(self.community_bounds)

        if not candidates:
            return []

        scores = {}
        mean_embedding = np.mean(
            [self.node_embeddings[n] for n in self.graph.nodes if n in self.node_embeddings], axis=0
        )

        pagerank = nx.pagerank(self.graph)

        for node in candidates:
            if node not in self.node_embeddings:
                continue
            deg = self.graph.degree(node)
            pr = pagerank.get(node, 0)
            l2 = -np.linalg.norm(self.node_embeddings[node] - mean_embedding)  # closer = higher score
            scores[node] = deg + pr * 100 + l2  # weighted sum

        sorted_nodes = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return [node for node, _ in sorted_nodes[:k]]



class EmbeddedGraph:
    def __init__(self, graph: nx.Graph):
        """
        Initializes the EmbeddedGraph with a NetworkX graph.

        Args:
            graph (nx.Graph): Input graph to embed and decompose into communities.
        """
        self.graph = graph.copy()  # Create a copy to avoid modifying the original
        self.community_to_nodes = {}
        self.node_to_community = {}
        self.node_embeddings = defaultdict(int)
        self.community_bounds = defaultdict(list)
        self.embedded_communities = {}
        self.community_graph = nx.Graph()

        self._run_pipeline()

    def _run_pipeline(self) -> None:
        """
        Internal method that runs the full embedding pipeline:
        Louvain → per-community node2vec → inter-edge analysis → community graph construction.
        """
        # Step 1: Detect communities using Louvain method
        self.community_to_nodes = self.run_louvain()

        # Build a lookup of node -> community_id
        for community_id, nodes in self.community_to_nodes.items():
            for node in nodes:
                self.node_to_community[node] = community_id

        # Find edges between different communities
        for u, v in self.graph.edges():
            if self.node_to_community.get(u) != self.node_to_community.get(v):
                self.community_bounds[self.node_to_community.get(u)].append(u)
                self.community_bounds[self.node_to_community.get(v)].append(v)

        # Step 2: Embed nodes within each community
        # Process each community separately
        for community_id, nodes in self.community_to_nodes.items():
            # Extract the subgraph for this community
            subgraph = self.graph.subgraph(nodes)
            # Skip communities that are too small for meaningful embedding
            if len(subgraph.nodes) < 2:
                continue
            embedded_community = EmbeddedCommunity(subgraph, self.community_bounds[community_id])
            embedded_community.embed()

            self.embedded_communities[community_id] = embedded_community
            self.node_embeddings.update(embedded_community.node_embeddings)
            #self.community_bounds[community_id] = embedded_community.get_top_k_boundary_nodes(k=5)


        # Step 5: Build community-level graph
        self.community_graph = self.construct_skeleton_graph()

    def run_louvain(self) -> Dict[int, List[Hashable]]:
        """
        Detects communities using the Louvain method.

        Returns:
            Dict[int, List[Hashable]]: Mapping from community ID to list of node IDs.
        """
        # Run Louvain community detection
        partition = community_louvain.best_partition(self.graph)

        # Group nodes by community
        communities = {}
        for node, community_id in partition.items():
            if community_id not in communities:
                communities[community_id] = []
            communities[community_id].append(node)

        return communities

    def construct_skeleton_graph(self):
        """
        Constructs a skeleton graph connecting boundary nodes across communities.

        Returns:
            nx.Graph: The skeleton graph
        """
        skeleton = nx.Graph()

        # Step 1: Add intra-community boundary node connections
        for community_id, boundary_nodes in self.community_bounds.items():
            for u, v in combinations(boundary_nodes, 2):
                if u in self.node_embeddings and v in self.node_embeddings:
                    weight = np.linalg.norm(self.node_embeddings[u] - self.node_embeddings[v])
                    skeleton.add_edge(u, v, weight=weight)

        # Step 2: Add inter-community edges from original graph (only if the edge crosses communities)
        for u, v in self.graph.edges():
            cu = self.node_to_community.get(u)
            cv = self.node_to_community.get(v)
            if cu is not None and cv is not None and cu != cv:
                skeleton.add_edge(u, v, weight=1)

        return skeleton


def l2_distance(vec1: np.ndarray, vec2: np.ndarray) -> float:
    return np.linalg.norm(vec1 - vec2)


def a_star_search(graph: nx.Graph, source: Any, destination: Any,
                  heuristic: Callable[[Any, Any], float] = l2_distance) -> Tuple[list, float]:
    def heuristic_wrapper(n1, _):
        return heuristic(n1, destination)

    try:
        path = nx.astar_path(graph, source, destination, heuristic=heuristic_wrapper, weight='weight')
        cost = nx.path_weight(graph, path, weight='weight')
        return path, cost
    except nx.NetworkXNoPath:
        return None, float('inf')
    except Exception as e:
        print(f"[A* ERROR] {e}")
        return None, float('inf')


class HierarchicalEmbeddedGraph:
    def __init__(self, graph: nx.Graph, max_nodes: int, num_levels: int):
        self.original_graph = graph
        self.num_levels = num_levels
        self.max_nodes = max_nodes
        self.embedded_graphs = []

        self._build_hierarchy()

        self.node_to_comm = defaultdict(dict)
        self.bitmask = defaultdict(int)
        self._build_indexing()
        self.ensure_edge_weights()

    def _build_hierarchy(self) -> None:
        current_graph = self.original_graph

        for level in range(self.num_levels):
            embedded_graph = EmbeddedGraph(current_graph)
            self.embedded_graphs.append(embedded_graph)

            if len(embedded_graph.community_graph.nodes) <= 5:
                break

            current_graph = embedded_graph.community_graph

        if self.embedded_graphs:
            first_level = self.embedded_graphs[0]
            self.graph = first_level.graph
            self.community_to_nodes = first_level.community_to_nodes
            self.node_embeddings = first_level.node_embeddings
            self.community_graph = first_level.community_graph

    def _build_indexing(self):
        for level, embedded_graph in enumerate(self.embedded_graphs):
            for node, comm_id in embedded_graph.node_to_community.items():
                self.node_to_comm[level][node] = comm_id
                bit_position = hash(comm_id) % 64
                self.bitmask[node] |= (1 << bit_position)

    def shared_level(self, u, v):
        if self.bitmask[u] & self.bitmask[v] == 0:
            return None

        for l in range(len(self.embedded_graphs)):
            if self.node_to_comm[l].get(u) == self.node_to_comm[l].get(v):
                return l

        return None

    def ensure_edge_weights(self, default_weight: float = 1.0, verbose: bool = True):
        for level, embedded in enumerate(self.embedded_graphs):
            graph = embedded.graph
            patched = 0
            for u, v in graph.edges():
                if 'weight' not in graph[u][v]:
                    graph[u][v]['weight'] = default_weight
                    patched += 1
            if verbose:
                print(f"[LEVEL {level}] Patched {patched} edges in `graph`.")

            community_graph = embedded.community_graph
            patched = 0
            for u, v in community_graph.edges():
                if 'weight' not in community_graph[u][v]:
                    community_graph[u][v]['weight'] = default_weight
                    patched += 1
            if verbose:
                print(f"[LEVEL {level}] Patched {patched} edges in `community_graph`.")

    def parallel_astar_to_boundaries(self, node, level=0, k=5):
        graph = self.embedded_graphs[level].graph
        embeddings = self.embedded_graphs[level].node_embeddings
        node_to_comm = self.embedded_graphs[level].node_to_community
        comm_id = node_to_comm.get(node)

        if comm_id is None:
            print(f"[PARALLEL] Node {node} not found in community map at level {level}")
            return {}

        boundaries = self.embedded_graphs[level].community_bounds.get(comm_id, [])[:k]
        results = {}

        def search_to_target(target):
            if node not in embeddings or target not in embeddings:
                print(f"[PARALLEL] Missing embeddings for node {node} or target {target}")
                return target, (None, float('inf'))

            def heuristic(u, _):
                return l2_distance(embeddings[u], embeddings[target])

            return target, a_star_search(graph, node, target, heuristic=heuristic)

        with ThreadPoolExecutor() as executor:
            future_to_target = {executor.submit(search_to_target, t): t for t in boundaries}
            for future in as_completed(future_to_target):
                target, result = future.result()
                results[target] = result
                print(f"[PARALLEL] {node} → {target}: {result}")

        return results

    def sequential_astar_to_boundaries(self, node, level=0, k=5):
        graph = self.embedded_graphs[level].graph
        embeddings = self.embedded_graphs[level].node_embeddings
        node_to_comm = self.embedded_graphs[level].node_to_community
        comm_id = node_to_comm.get(node)

        if comm_id is None:
            print(f"[SEQUENTIAL] Node {node} not found in community map at level {level}")
            return {}

        boundaries = self.embedded_graphs[level].community_bounds.get(comm_id, [])[:k]
        results = {}

        for target in boundaries:
            if node not in embeddings or target not in embeddings:
                print(f"[SEQUENTIAL] Missing embeddings for node {node} or target {target}")
                results[target] = (None, float('inf'))
                continue

            def heuristic(u, _):
                return l2_distance(embeddings[u], embeddings[target])

            path, cost = a_star_search(graph, node, target, heuristic=heuristic)
            results[target] = (path, cost)
            print(f"[SEQUENTIAL] {node} → {target}: {path}, cost={cost}")

        return results


In [ ]:
# Now that the full implementation is defined, we can run the benchmark using the 'g' graph

import time
import numpy as np
import networkx as nx
from typing import List, Tuple

def generate_random_node_pairs(graph: nx.Graph, num_pairs: int = 10) -> List[Tuple[int, int]]:
    nodes = list(graph.nodes())
    pairs = []
    while len(pairs) < num_pairs:
        src, dst = np.random.choice(nodes, 2, replace=False)
        if nx.has_path(graph, src, dst):
            pairs.append((src, dst))
    return pairs

def benchmark_astar_vs_networkx(h_graph, graph: nx.Graph, node_pairs: List[Tuple[int, int]],
                                 level=0, k=5, min_length=3):
    parallel_total_time = 0.0
    networkx_total_time = 0.0
    parallel_calls = 0
    dijkstra_calls = 0

    for src, _ in node_pairs:
        comm_map = h_graph.embedded_graphs[level].node_to_community
        comm_id = comm_map.get(src)
        if comm_id is None:
            continue

        boundaries = h_graph.embedded_graphs[level].community_bounds.get(comm_id, [])
        filtered_targets = []

        for target in boundaries:
            if src == target:
                continue
            try:
                path = nx.shortest_path(graph, src, target)
                if len(path) - 1 >= min_length:
                    filtered_targets.append(target)
            except nx.NetworkXNoPath:
                continue

            if len(filtered_targets) >= k:
                break

        if len(filtered_targets) < k:
            continue

        h_graph.embedded_graphs[level].community_bounds[comm_id] = filtered_targets

        # Parallel A*
        start = time.time()
        _ = h_graph.parallel_astar_to_boundaries(src, level=level, k=k)
        end = time.time()
        parallel_total_time += (end - start)
        parallel_calls += k

        # NetworkX Dijkstra for each target
        for target in filtered_targets:
            start = time.time()
            try:
                _ = nx.dijkstra_path(graph, src, target)
                networkx_total_time += time.time() - start
                dijkstra_calls += 1
            except nx.NetworkXNoPath:
                continue

    return parallel_total_time / max(parallel_calls, 1), networkx_total_time / max(dijkstra_calls, 1)



def summarize_results(parallel_times, networkx_times):
    def summarize(name, times):
        print(f"{name} AVERAGE TIME: {np.mean(times):.6f}s over {len(times)} runs")
    summarize("Parallel A*", parallel_times)
    summarize("NetworkX Dijkstra", networkx_times)

# Assuming 'g' is preloaded Citeseer graph
#h_graph = HierarchicalEmbeddedGraph(g, max_nodes=500, num_levels=2)
#h_graph.ensure_edge_weights()
h_graph = heg
g = G
#pairs = generate_random_node_pairs(G, num_pairs=100)
pairs = generate_random_node_pairs(g, num_pairs=100)
p_avg, nx_avg = benchmark_astar_vs_networkx(h_graph, g, pairs, level=0, k=10, min_length=5)

print(f"Parallel A* Avg Time per Path: {p_avg:.6f}s")
print(f"NetworkX Dijkstra Avg Time per Path: {nx_avg:.6f}s")




[PARALLEL] 1022 → 1033: ([1022, 1461, 2918, 1710, 2112, 1033], 5.0)
[PARALLEL] 1022 → 2816: ([1022, 1461, 2918, 1710, 2112, 1033, 411, 1410, 1324, 2816], 9.0)
[PARALLEL] 1022 → 411: ([1022, 1461, 2918, 1710, 2112, 1033, 411], 6.0)
[PARALLEL] 1022 → 430: ([1022, 1461, 2918, 1710, 2112, 1033, 2819, 1992, 455, 1684, 430], 10.0)
[PARALLEL] 1022 → 455: ([1022, 1461, 2918, 1710, 2112, 1033, 2819, 1992, 455], 8.0)
[PARALLEL] 1022 → 1865: ([1022, 1461, 2215, 1710, 2112, 1307, 1865], 6.0)
[PARALLEL] 1022 → 2973: ([1022, 1461, 2918, 1710, 2112, 1033, 411, 1410, 2345, 465, 2973], 10.0)
[PARALLEL] 1022 → 1992: ([1022, 1461, 2918, 1710, 2112, 1033, 2819, 1992], 7.0)
[PARALLEL] 1022 → 1410: ([1022, 1461, 2918, 1710, 2112, 1033, 411, 1410], 7.0)
[PARALLEL] 1022 → 2819: ([1022, 1461, 2918, 1710, 2112, 1033, 2819], 6.0)
Parallel A* Avg Time per Path: 0.000914s
NetworkX Dijkstra Avg Time per Path: 0.000341s


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

# Load the CiteSeer dataset
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer', transform=T.NormalizeFeatures())
data = dataset[0]  # The graph object


from torch_geometric.utils import to_networkx

# Convert to undirected NetworkX graph
G = to_networkx(data, to_undirected=True)

print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")



Graph has 3327 nodes and 4552 edges.


In [ ]:
def debug_node_to_boundary(heg: HierarchicalEmbeddedGraph, node: Any, level: int = 0):
    print(f"\n🔍 Debugging node {node} at level {level}...")

    graph = heg.embedded_graphs[level].graph
    embeddings = heg.embedded_graphs[level].node_embeddings
    node_to_comm = heg.embedded_graphs[level].node_to_community
    comm_bounds = heg.embedded_graphs[level].community_bounds

    # Check embedding
    if node not in embeddings:
        print("❌ Node has no embedding.")
    else:
        print("✅ Node has an embedding.")

    # Check community
    comm = node_to_comm.get(node)
    if comm is None:
        print("❌ Node has no community assignment.")
        return
    else:
        print(f"✅ Node is in community {comm}")

    boundaries = comm_bounds.get(comm, [])
    print(f"📍 Top-k boundary nodes for this community: {boundaries}")

    for boundary in boundaries:
        print(f"\n--- Checking path to boundary node {boundary} ---")
        if boundary not in embeddings:
            print("⚠️  Boundary node has no embedding.")
        if not nx.has_path(graph, node, boundary):
            print("🚫 No path in the graph.")
        else:
            print("✅ Path exists in graph.")
            path = nx.shortest_path(graph, node, boundary)
            print(f"🧭 Path: {path}, Length: {len(path)-1}")
            dist = l2_distance(embeddings.get(node, np.zeros(128)), embeddings.get(boundary, np.zeros(128)))
            print(f"📐 Embedding L2 distance: {dist:.4f}")
debug_node_to_boundary(heg, 1424)



🔍 Debugging node 1424 at level 0...
✅ Node has an embedding.
✅ Node is in community 35
📍 Top-k boundary nodes for this community: [2174, 2861, 2228, 49, 1253]

--- Checking path to boundary node 2174 ---
✅ Path exists in graph.
🧭 Path: [1424, 709, 2174], Length: 2
📐 Embedding L2 distance: 2.3412

--- Checking path to boundary node 2861 ---
✅ Path exists in graph.
🧭 Path: [1424, 709, 1471, 2861], Length: 3
📐 Embedding L2 distance: 2.5385

--- Checking path to boundary node 2228 ---
✅ Path exists in graph.
🧭 Path: [1424, 709, 1471, 682, 2228], Length: 4
📐 Embedding L2 distance: 2.9976

--- Checking path to boundary node 49 ---
✅ Path exists in graph.
🧭 Path: [1424, 709, 1471, 682, 2228, 49], Length: 5
📐 Embedding L2 distance: 3.0793

--- Checking path to boundary node 1253 ---
✅ Path exists in graph.
🧭 Path: [1424, 709, 1471, 2861, 2455, 1253], Length: 5
📐 Embedding L2 distance: 3.3356


In [ ]:
!pip install ogb
!pip install node2vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import time
import random
import numpy as np
import networkx as nx

from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.utils import to_networkx

# === Load ogbn-arxiv Dataset and Build Graph ===
dataset = PygNodePropPredDataset(name='ogbn-arxiv', root='data')
data = dataset[0]
G = to_networkx(data, to_undirected=True)

# === Create Hierarchical Embedded Graph ===
h_graph = HierarchicalEmbeddedGraph(G, max_nodes=500, num_levels=3)


# === Sample Long-Distance Pairs ===
def sample_distant_node_pairs(graph: nx.Graph, num_pairs: int = 20, min_distance: int = 5):
    """
    Samples node pairs at least `min_distance` apart.
    """
    nodes = list(graph.nodes)
    pairs = []
    attempts = 0
    while len(pairs) < num_pairs and attempts < num_pairs * 10:
        u, v = random.sample(nodes, 2)
        try:
            d = nx.shortest_path_length(graph, source=u, target=v)
            if d >= min_distance:
                pairs.append((u, v))
        except nx.NetworkXNoPath:
            pass
        attempts += 1
    return pairs


# === Benchmark Latency Only ===
def benchmark_latency_only(graph, h_graph, pairs):
    results = []
    for src, dst in pairs:
        # Hierarchical
        start = time.perf_counter()
        h_path, h_dist = h_graph.find_hierarchical_path(src, dst)
        h_time = time.perf_counter() - start

        # NetworkX Dijkstra
        try:
            start = time.perf_counter()
            nx_dist = nx.dijkstra_path_length(graph, src, dst)
            nx_time = time.perf_counter() - start
        except nx.NetworkXNoPath:
            nx_dist = float('inf')
            nx_time = time.perf_counter() - start

        results.append((src, dst, h_time, nx_time, h_dist, nx_dist))
    return results


# === Print Benchmark Summary ===
def print_latency_summary(results):
    h_times = [r[2] for r in results]
    nx_times = [r[3] for r in results]

    print("\n--- Latency Benchmark Summary ---")
    print(f"Queries Run: {len(results)}")
    print(f"Avg Hierarchical Query Time: {np.mean(h_times):.4f}s")
    print(f"Avg NetworkX Dijkstra Time:  {np.mean(nx_times):.4f}s")
    print("\nDetails per Query:")
    for r in results:
        print(f"Src: {r[0]}, Dst: {r[1]} | H: {r[2]:.4f}s | NX: {r[3]:.4f}s | H_dist: {r[4]:.2f} | NX_dist: {r[5]:.2f}")


# === Run Everything ===
if __name__ == "__main__":
    print("Sampling distant node pairs...")
    pairs = sample_distant_node_pairs(G, num_pairs=20, min_distance=6)

    print("Running latency benchmark...")
    results = benchmark_latency_only(G, h_graph, pairs)

    print_latency_summary(results)


KeyboardInterrupt: 

In [ ]:
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s eta 0:00:00


In [ ]:
# Step 1: Clean reinstall numpy and gensim to fix binary incompatibility
!pip install --force-reinstall --no-cache-dir numpy
!pip install --force-reinstall --no-cache-dir gensim
!pip install --force-reinstall --no-cache-dir node2vec
!pip install --force-reinstall --no-cache-dir python-louvain

# Step 2: Restart runtime to clear out the C-extension issues
import os
os.kill(os.getpid(), 9)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 291.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 226.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 186.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 183.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 175.3 MB/s eta 0:00:00
   ━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 220.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 317.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 286.9 MB/s eta 0:00:00
